In [109]:
# coding=utf-8
import math
from scapy.all import *
import matplotlib.pyplot as plt
import numpy as np
import time


def enviar_paquete(host, time_to_live):
    packet = IP(dst=host, ttl=time_to_live) / ICMP()
    res = sr(packet, timeout=2, verbose=0)
    return res[0][ICMP]

In [113]:
def trace_route2(host):
    hops = []
    tipo = 11
    ttl = 1
    origen = 0
    while (tipo!=0):
        ti = time.time()
        res1 = enviar_paquete(host, ttl)
        tf = time.time()
        rtt = tf-ti
        print ti, tf, rtt
        if (len(res1) > 0):
            tipo = res1[0][1].type
            src = res1[0][1].src
            print "hop = %d rtt = %d ICMP type = %d" % (ttl,rtt, tipo)
            if (tipo > 0):
                hops.append(src)
            else:
                hops.append(host + ' at ' + src)
        else:
            print 'ttl = ', ttl, " timeout"
        ttl = ttl + 1
    return hops
        #print "src: " +  + " dst:" + res1[0][1].dst + " tipo: %d" % res1[0][1].type

In [114]:
trace_route2('www.google.com')

1478247116.49 1478247118.68 2.19055294991
ttl =  1  timeout
1478247118.68 1478247120.75 2.07049489021
ttl =  2  timeout
1478247120.76 1478247122.83 2.07859611511
ttl =  3  timeout
1478247122.83 1478247124.91 2.07929301262
ttl =  4  timeout
1478247124.92 1478247126.97 2.05919504166
ttl =  5  timeout
1478247126.98 1478247127.07 0.0946800708771
hop = 6 rtt = 0 ICMP type = 11
1478247127.07 1478247127.15 0.083596944809
hop = 7 rtt = 0 ICMP type = 11
1478247127.15 1478247129.21 2.05975198746
ttl =  8  timeout
1478247129.22 1478247129.29 0.0749459266663
hop = 9 rtt = 0 ICMP type = 0


['200.89.165.197', '200.89.160.14', 'www.google.com at 181.30.242.173']

In [7]:
enviar_paquete("google.com",1)

Begin emission:
Finished to send 1 packets.

Received 126 packets, got 0 answers, remaining 1 packets


<ICMP from Results: TCP:0 UDP:0 ICMP:0 Other:0>

In [90]:
host = 'www.google.com'
time_to_live = 58
packet = IP(dst=host, ttl=time_to_live) / ICMP()
res = sr(packet)
res[0][ICMP][0][1]

Begin emission:
Finished to send 1 packets.

Received 44 packets, got 1 answers, remaining 0 packets


<IP  version=4L ihl=5L tos=0x3 len=28 id=62826 flags= frag=0L ttl=58 proto=icmp chksum=0x2197 src=181.30.242.177 dst=192.168.1.100 options=[] |<ICMP  type=echo-reply code=0 chksum=0xffff id=0x0 seq=0x0 |<Padding  load='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>>

In [50]:
len(enviar_paquete('www.google.com',1))

Begin emission:
Finished to send 1 packets.

Received 44 packets, got 0 answers, remaining 1 packets


0

In [82]:
print "hop %d rtt %d " % (1,2)

hop 1 rtt 2 


In [133]:
answer = sr1(IP(dst="8.8.8.8")/UDP(dport=53)/DNS(rd=1, qd=DNSQR(qname="www.google.com", qclass="IN", qtype="A")),verbose=0)
answer[DNS]

<DNS  id=0 qr=1L opcode=QUERY aa=0L tc=0L rd=1L ra=1L z=0L ad=0L cd=0L rcode=ok qdcount=1 ancount=1 nscount=0 arcount=0 qd=<DNSQR  qname='www.google.com.' qtype=A qclass=IN |> an=<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=199 rdata='172.217.28.4' |> ns=None ar=None |>

In [116]:
DNS?